In [1]:
import os
import ee
import geemap
import pandas as pd

In [2]:
data = pd.read_csv("GHG_Data_Sample.csv")

In [3]:
data['Date'] = pd.to_datetime(data['Date'], format = '%m/%d/%y')
data.head()

,Date,Site,ID,Lat,Long,CO2PixelDay
0,2015-07-15,Goodrich,R1,40.366021,-120.949558,80.613691
1,2015-07-15,Goodrich,R10,40.365042,-120.949802,82.194613
2,2015-07-15,Goodrich,R11,40.365238,-120.950044,44.048585
3,2015-07-15,Goodrich,R12,40.365436,-120.950289,94.443643
4,2015-07-15,Goodrich,R2,40.365824,-120.949316,157.806186


In [4]:
#ee.Authenticate()
ee.Initialize()

In [38]:
x, y = data.loc[1, ['Long', 'Lat']]
point = ee.Geometry.Point(x, y)

In [39]:
target_date = data.loc[1, 'Date'].strftime('%Y-%m-%d')

In [63]:
def maskClouds(image):
    quality = image.select('pixel_qa')
    cloud = quality.bitwiseAnd(1 << 5).eq(0)    # mask out cloud shadow
    clear = quality.bitwiseAnd(1 << 4).eq(0)     # mask out cloud
    return image.updateMask(cloud).updateMask(clear)

In [41]:
def calculateCloudScore(image):
    cloud = ee.Algorithms.Landsat.simpleCloudScore(image).select('cloud')
    return image.addBands(cloud)

In [79]:
landsat_collection = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR')

In [81]:
spatial_filtered = landsat_collection.filterBounds(point)
temporal_filtered = spatial_filtered.filterDate(ee.Date(target_date).advance(-7, 'day'), ee.Date(target_date).advance(7, 'day'))

In [82]:
cloud_free_images = temporal_filtered.map(maskClouds)#.map(calculateCloudScore)

In [83]:
sorted_collection = cloud_free_images.sort('cloud')

In [84]:
nearest_image = ee.Image(sorted_collection.first())

In [85]:
nearest_image

In [70]:
bands = nearest_image.select(['B2', 'B3', 'B4', 'B5', 'B6', 'B7'])

In [71]:
bands

In [72]:
band_values = bands.reduceRegion(ee.Reducer.mean(), point, 30).getInfo()

In [73]:
print('Band Values:', band_values)

Band Values: {'B2': 568, 'B3': 763, 'B4': 865, 'B5': 2008, 'B6': 2762, 'B7': 1994}


In [52]:
map_l8 = geemap.Map(center=[y,x], zoom=10)
image_viz_params = {'bands': ['B5', 'B4', 'B3'], 'min': 0, 'max': 0.5, 'gamma': [0.95, 1.1, 1]}

# Add the image layer to the map and display it.
map_l8.add_layer(nearest_image, image_viz_params, 'false color composite')
display(map_l8)

Map(center=[40.36504242, -120.9498018], controls=(WidgetControl(options=['position', 'transparent_bg'], widget…